# **TRAIN A CUSTOM YOLOv4 OBJECT DETECTOR**


# **In this tutorial, we will be training a custom detector for mask detection using YOLOv4 and Darknet**

#**HOW TO BEGIN**?

* Click on **File** in the menu bar and click on **Save a copy in drive**.This will open a copy of my colab notebook on your browser which you can now use.

* Next, once you have opened the copy of my notebook and are connected to the Google Colab VM , click on **Runtime** in the menu bar and click on **Change runtime type**. Select **GPU** and click on save.


# **1) Create *`yoloV4-ppe-detection-model`* and *`training`* and *`test`*  folders in your drive**

 Create a folder named ***yoloV4-ppe-detection-model*** in your disk. 
 
 Next, create another folder named ***training*** and ***test*** inside the ***yoloV4-ppe-detection-model*** folder. This is where we will save our trained weights (This path is mentioned in the ***obj.data*** file which we will upload later) 


# **2) Mount drive, link your folder and navigate to /mydrive/yolov4 folder**

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive

# list the contents of /folder
!ls ../yoloV4-ppe-detection-model

#Navigate to /mydrive/yoloV4-ppe-detection-model
%cd ../yoloV4-ppe-detection-model

# **3) Clone `darknet` git repository** 

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

# **4) Create & upload the following files which we need for training a custom detector**


a.   `Labeled Custom Dataset` 

b.   `Custom cfg file`

c.   `obj.data and obj.names files` 

d.   `process.py file (to create train.txt and test.txt files for training)`




## **4(a) Upload the Labeled custom dataset *`obj.zip`* file to the *`yolov4`* folder on your drive**


Create the zip file **obj.zip** from the **obj** folder containing both the input image ".jpg" files and their corresponding YOLO format labeled ".txt" files. 

Upload the zip file to the ***yoloV4-ppe-detection-model*** folder on your drive.

## **4(b) Create your custom *`config`* file and upload it to your drive**

Download the **yolov4-custom.cfg** file from ***darknet/cfg*** directory, make changes to it, and upload it to the ***yolov4*** folder on your drive .

You can also download the custom config file from the official [AlexeyAB Github](https://www.github.com/AlexeyAB/darknet)


**You need to make the following changes in your custom config file:**

*   change line batch to batch=64
*   change line subdivisions to subdivisions=16
*   change line max_batches to (classes*2000 but not less than number of training images and not less than 6000), f.e. max_batches=6000 if you train for 3 classes
*    change line steps to 80% and 90% of max_batches, f.e. steps=4800,5400
*    set network size width=416 height=416 or any value multiple of 32
*    change line classes=80 to your number of objects in each of 3 [yolo]-layers
*    change [filters=255] to filters=(classes + 5)x3 in the 3 [convolutional] before each [yolo] layer, keep in mind that it only has to be the last [convolutional] before each of the [yolo] layers.
     So if classes=1 then it should be filters=18. If classes=2 then write filters=21.



## **4(c) Create your *`obj.data`* and *`obj.names`* files and upload to your drive**



### **obj.data**

```
classes = 2
train  = data/train.txt
valid  = data/test.txt
names = data/obj.names
backup = /mydrive/yolov4/training

```

## **4(d) Upload the *`process.py`* script file to the *`yolov4`* folder on your drive**

**To divide all image files into 2 parts. 90% for train and 10% for test.**

This *`process.py`* script creates the files *`train.txt`* & *`test.txt`* where the *`train.txt`* file has paths to 90% of the images and *`test.txt`* has paths to 10% of the images.


# **5) Make changes in the `makefile` to enable OPENCV and GPU**

In [ ]:
# change makefile to have GPU and OPENCV enabled
# also set CUDNN, CUDNN_HALF and LIBSO to 1

%cd ../yoloV4-ppe-detection-model/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
!echo $PATH

# **6) Run `make` command to build darknet**

In [ ]:
# build darknet 
!make

# **7) Copy all the files from the *`yolov4`* folder to the darknet directory**

In [1]:
# Clean the data and cfg folders first except the labels folder in data which is required

%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
%cd ..

%rm -rf cfg/
%mkdir cfg

[Errno 2] No such file or directory: 'data/'
/home/rushikesh/Documents/RK
/home/rushikesh/Documents


# unzip data in to data folder

In [ ]:
# Unzip the obj.zip dataset and its contents so that they are now in /darknet/data/ folder 
!unzip ../yoloV4-ppe-detection-model/obj.zip -d data/

In [ ]:
# Copy the yolov4-custom.cfg file so that it is now in /darknet/cfg/ folder 

!cp ../yoloV4-ppe-detection-model/yolov4-custom.cfg cfg

# verify if your custom file is in cfg folder
!ls cfg/

In [ ]:
# Copy the obj.names and obj.data files from your drive so that they are now in /darknet/data/ folder 

!cp ../yoloV4-ppe-detection-model/obj.names data
!cp ../yoloV4-ppe-detection-model/obj.data  data

# verify if the above files are in data folder
!ls data/

In [ ]:
# Copy the process.py file to the current darknet directory 

!cp ../yoloV4-ppe-detection-model/process.py .

# **8) Run the *`process.py`* python script to create the *`train.txt`* & *`test.txt`* files inside the *data* folder**

In [ ]:
# run process.py ( this creates the train.txt and test.txt files in our darknet/data folder )
!python process.py

# list the contents of data folder to check if the train.txt and test.txt files have been created 
!ls data/

# **9) Download the pre-trained *`yolov4`* weights**

In [ ]:
# Download the yolov4 pre-trained weights file
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# **10) TRAINING**

## **Train your custom detector** 

For best results, you should stop the training when the average loss is less than 0.05 if possible or at least below 0.3, else train the model until the average loss does not show any significant change for a while.

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
    
!./darknet detector train data/obj.data cfg/yolov4-custom.cfg yolov4.conv.137 -map

### **<ins>NOTE</ins>:** If you get disconnected or lose your session for some reason you have to run steps 2, 5, and 6 again to mount the drive, edit makefile and build darknet every single time, otherwise the darknet executable will not work.

## **To restart your training (In case the training does not finish and you get disconnected)**

If you get disconnected or lose your session, you don't have to start training your model from scratch again. You can restart training from where you left off. Use the weights that were saved last. The weights are saved every 100 iterations as ***yolov4-custom_last.weights*** in the ***yoloV4-ppe-detection-model/training*** folder on your drive. (The path we gave as backup in "obj.data" file).


### **So to restart training run only Steps 2, 5, 6 and then run the following command**

In [ ]:
#to restart training your custom detector where you left off(using the weights that were saved last)

!./darknet detector train data/obj.data cfg/yolov4-custom.cfg ../yoloV4-ppe-detection-model/training/yolov4-custom_last.weights -dont_show -map


# **11) Check performance** 



In [ ]:
# define helper function imShow
def imShow(path):
    import cv2
    import matplotlib.pyplot as plt
    %matplotlib inline

    image = cv2.imread(path)
    height, width = image.shape[:2]
    resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

    fig = plt.gcf()
    fig.set_size_inches(18, 10)
    plt.axis("off")
    plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
    plt.show()


**Check the training chart**

In [ ]:
#only works if the training does not get interrupted
imShow('chart.png')


**Check mAP (mean average precision)**

In [ ]:
##You can check the mAP for all the saved weights to see which gives the best results ( xxxx here is the saved weight number like 4000, 5000 or 6000 snd so on )

!./darknet detector map data/obj.data cfg/yolov4-custom.cfg ../yoloV4-ppe-detection-model/training/yolov4-custom_last.weights -points 0

# **12) Test your custom Object Detector**

## **Make changes to your custom config file**
*   change line batch to batch=1
*   change line subdivisions to subdivisions=1

You can do it either manually or by simply running the code below

In [ ]:
#set your custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-custom.cfg
%cd ..

## **Run detector on an image**

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, the thresh flag sets the minimum accuracy required for object detection)

!./darknet detector test data/obj.data cfg/yolov4-custom.cfg ../yoloV4-ppe-detection-model/training/yolov4-custom_last.weights ../yoloV4-ppe-detection-model/test/img.jpg -i 0 -thresh 0.7
imShow('predictions.jpg')